In [1]:
# === 1. SETUP ===
import os
import tarfile
import numpy as np
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import make_grid

In [4]:
# === 2. EXTRACT THE UPLOADED DATASET (Colab) ===
from google.colab import files
import os
import tarfile

# Upload the dataset manually
uploaded = files.upload()  # Upload 102flowers.tgz from your machine

# Assume it was uploaded as '102flowers.tgz'
tgz_path = "/content/102flowers.tgz"
extract_path = "/content/102flowers"

# Extract the archive
with tarfile.open(tgz_path, "r:gz") as tar:
    tar.extractall(path=extract_path)

# Fix nested folder structure if needed
image_root = os.path.join(extract_path, "jpg")


Saving 102flowers.tgz to 102flowers (1).tgz


In [5]:
# === 3. IMAGE TRANSFORMS ===
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [7]:
import os
import shutil
from glob import glob
from random import randint

# Simulate class folders: create folders class_1 to class_102
simulated_root = "/content/flowers_dataset"
os.makedirs(simulated_root, exist_ok=True)

for i in range(1, 103):
    os.makedirs(os.path.join(simulated_root, f"class_{i}"), exist_ok=True)

# Move each image to a random class folder (demo purpose only)
image_files = glob(os.path.join(image_root, "*.jpg"))

for img_path in image_files:
    class_id = randint(1, 102)
    dest_folder = os.path.join(simulated_root, f"class_{class_id}")
    shutil.copy(img_path, dest_folder)

# Now image_root points to a folder with 102 subfolders
image_root = simulated_root


In [8]:
# === 5. LOAD PRETRAINED MODEL (ResNet-18) ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers

# Replace final layer for 102 flower classes
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 102)
)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 126MB/s]


In [9]:
# === 6. TRAINING SETUP ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3)

In [10]:
# === 7. TRAIN FUNCTION WITH OPTIONAL EARLY STOPPING ===
def train_model(model, num_epochs=100, early_stop=True, patience=10):
    best_loss = float('inf')
    wait = 0
    train_losses, val_losses = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {correct / total:.4f}")

        # Early stopping logic
        if early_stop:
            if val_loss < best_loss:
                best_loss = val_loss
                wait = 0
                torch.save(model.state_dict(), "best_model.pth")
            else:
                wait += 1
                if wait >= patience:
                    print("Early stopping triggered.")
                    break

    return train_losses, val_losses

In [13]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

# Define your transforms (if not already defined)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

# Point to the folder with class subdirectories
image_root = "/content/flowers_dataset"

# Load dataset
full_dataset = ImageFolder(image_root, transform=train_transforms)


In [14]:
# Step 6
from torch.utils.data import random_split

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


In [16]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import copy

def train_model(model, num_epochs=50, early_stop=True, patience=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    patience_counter = 0

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

        epoch_val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)
        scheduler.step(epoch_val_loss)

        print(f"Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")

        # Early stopping
        if early_stop:
            if epoch_val_loss < best_loss:
                best_loss = epoch_val_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping triggered.")
                    break

    model.load_state_dict(best_model_wts)
    return train_losses, val_losses


In [ ]:
# Example: Load pretrained model and customize
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)

# Modify final layer for 102 classes
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_features, 102)
)
model = model.to(device)

# Train
train_losses, val_losses = train_model(model, num_epochs=100, early_stop=True, patience=10)


Epoch 1/100
Train Loss: 4.9037 | Val Loss: 4.6810
Epoch 2/100
